Hello everyone!

This is a slightly modification from [Jeremy Howard's great kernel](https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline), using [ClassifierChain](http://scikit-learn.org/stable/auto_examples/multioutput/plot_classifier_chain_yeast.html) from scikit-learn.

It's a bit late for a baseline right now, but maybe you can `b l e n d` it in your mix!

Sorry for the mess, I made this very quickly.

In [25]:
import numpy as np
import pandas as pd
import re, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.pipeline import Pipeline
from scipy import sparse

In [46]:
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs
        self.classes_ = [0,1]

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
        return self

In [27]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [28]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/sample_submission.csv')

In [30]:
train['comment_text'].fillna('unknown', inplace=True)
test['comment_text'].fillna('unknown', inplace=True)

In [31]:
def tokenize(s):
    pattern = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
    return pattern.sub(r' \1 ', s).split()

In [32]:
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
X_train = vec.fit_transform(train['comment_text'])
X_test = vec.transform(test['comment_text'])

In [36]:
y_train = train[labels].values

In [48]:
chains = [ClassifierChain(NbSvmClassifier(), order='random', random_state=i) for i in range(len(labels))]

In [49]:
for chain in chains:
    chain.fit(X_train, y_train)

In [55]:
y_pred_chains = np.array([chain.predict_proba(X_test) for chain in chains])

In [56]:
y_pred_ensemble = y_pred_chains.mean(axis=0)

In [57]:
sub[labels] = y_pred_ensemble

In [59]:
sub.head()

In [60]:
sub.to_csv('nbsvm_chain.csv', index=False)